In [1]:
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

In [2]:
import time
import pandas as pd
from polygon import RESTClient
import sqlite3
import pandas as pd
import datetime
from pytz import timezone

In [3]:
client = RESTClient("beBybSi8daPgsTp5yx5cHtHpYcrjp5Jq")  # 使用你的API密钥
Final_df_1 = pd.DataFrame()
Final_df_2 = pd.DataFrame()
Final_df_3 = pd.DataFrame()

loop = 0
while loop < 50:
    loop += 1
    Auxiliary_df = pd.DataFrame(columns=["timestamp"])  # 创建一个空的DataFrame来存储数据

    start_time = time.time()  # 获取当前时间戳
    end_time = start_time + 6 * 60  # 持续运行6分钟
    i = 0
    last_time = start_time
    while time.time() < end_time:
        print('loop=', loop)
        print(i)
        i += 1
        rate1 = client.get_real_time_currency_conversion("EUR", "USD", precision=6)
        rate2 = client.get_real_time_currency_conversion("EUR", "AUD", precision=6)
        rate3 = client.get_real_time_currency_conversion("EUR", "CAD", precision=6)

        # 获取当前时间戳和汇率值
        timestamp_1 = rate1.last.timestamp
        timestamp_2 = rate2.last.timestamp
        timestamp_3 = rate3.last.timestamp
        dt_1 = datetime.datetime.fromtimestamp(timestamp_1 / 1000.0)
        dt_2 = datetime.datetime.fromtimestamp(timestamp_2 / 1000.0)
        dt_3 = datetime.datetime.fromtimestamp(timestamp_3 / 1000.0)
        rate_value1 = round(rate1.converted, 5)  # 或者 rate.conversion_rate
        rate_value2 = round(rate2.converted, 5)  # 或者 rate.conversion_rate
        rate_value3 = round(rate3.converted, 5)  # 或者 rate.conversion_rate

        # 将数据添加到DataFrame中
        Auxiliary_df = Auxiliary_df.append({"times_1": dt_1,"times_2": dt_2,"times_3": dt_3,
                                            "timestamp_1": timestamp_1,"timestamp_2": timestamp_2,"timestamp_3": timestamp_3,
                                            "rate1_pair": 'EUR2USD',"rate1": rate_value1, 
                                            "rate2_pair": 'EUR2AUD', "rate2": rate_value2,
                                            "rate3_pair": 'EUR2CAD', "rate3": rate_value3}, ignore_index=True)

        # 每秒钟运行一次，但是需要减去运行代码的时间
        sleep_time = max(0, 1 - (time.time() - last_time))
        time.sleep(sleep_time)
        last_time = time.time()

    
    # 连接到SQLite数据库
    conn = sqlite3.connect('Auxiliary_df.db')
    # 将DataFrame写入SQLite数据库中的表
    Auxiliary_df.to_sql('Auxiliary_df', conn, if_exists='append', index=False)
    # 关闭数据库连接
    conn.close()

    # 获取列的最大值
    max_rate1 = Auxiliary_df['rate1'].max()
    max_rate2 = Auxiliary_df['rate2'].max()
    max_rate3 = Auxiliary_df['rate3'].max()
    # 获取列的最小值
    min_rate1 = Auxiliary_df['rate1'].min()
    min_rate2 = Auxiliary_df['rate2'].min()
    min_rate3 = Auxiliary_df['rate3'].min()
    # 获取列的平均值
    mean_rate1 = Auxiliary_df['rate1'].mean()
    mean_rate2 = Auxiliary_df['rate2'].mean()
    mean_rate3 = Auxiliary_df['rate3'].mean()
    # 获取VOL
    VOL_rate1 = (max_rate1 - min_rate1) / mean_rate1
    VOL_rate2 = (max_rate2 - min_rate2) / mean_rate2
    VOL_rate3 = (max_rate3 - min_rate3) / mean_rate3


    if loop >= 2:
        crosses_pair_1 = 0
        crosses_pair_2 = 0
        crosses_pair_3 = 0
        # 创建一个新的列，代表前一行的价格数据，以便我们能判断穿线的情况
        Auxiliary_df['prev_rate1'] = Auxiliary_df['rate1'].shift(1)
        Auxiliary_df['prev_rate2'] = Auxiliary_df['rate2'].shift(1)
        Auxiliary_df['prev_rate3'] = Auxiliary_df['rate3'].shift(1)
        for band in keltner_bands_1:
            # 找出价格从下方穿过 Keltner band 的情况
            crosses_above = ((Auxiliary_df['prev_rate1'] < band) & (Auxiliary_df['rate1'] >= band))
            # 找出价格从上方穿过 Keltner band 的情况
            crosses_below = ((Auxiliary_df['prev_rate1'] > band) & (Auxiliary_df['rate1'] <= band))
            # 将这两种情况合并
            crosses_band = crosses_above | crosses_below
            # 累加穿过 Keltner band 的次数
            crosses_pair_1 += crosses_band.sum()
            
        for band in keltner_bands_2:
            # 找出价格从下方穿过 Keltner band 的情况
            crosses_above = ((Auxiliary_df['prev_rate2'] < band) & (Auxiliary_df['rate2'] >= band))
            # 找出价格从上方穿过 Keltner band 的情况
            crosses_below = ((Auxiliary_df['prev_rate2'] > band) & (Auxiliary_df['rate2'] <= band))
            # 将这两种情况合并
            crosses_band = crosses_above | crosses_below
            # 累加穿过 Keltner band 的次数
            crosses_pair_2 += crosses_band.sum()
            
        for band in keltner_bands_3:
            # 找出价格从下方穿过 Keltner band 的情况
            crosses_above = ((Auxiliary_df['prev_rate3'] < band) & (Auxiliary_df['rate3'] >= band))
            # 找出价格从上方穿过 Keltner band 的情况
            crosses_below = ((Auxiliary_df['prev_rate3'] > band) & (Auxiliary_df['rate3'] <= band))
            # 将这两种情况合并
            crosses_band = crosses_above | crosses_below
            # 累加穿过 Keltner band 的次数
            crosses_pair_3 += crosses_band.sum()

        # 获取FD
        FD1 = crosses_pair_1 / (max_rate1 - min_rate1)
        FD2 = crosses_pair_2 / (max_rate2 - min_rate2)
        FD3 = crosses_pair_3 / (max_rate3 - min_rate3)
        Final_df_1 = Final_df_1.append({"times_1": dt_1,
                                        "timestamp_1": timestamp_1,
                                        "max_rate1": max_rate1,
                                        "min_rate1": min_rate1, 
                                        "mean_rate1": mean_rate1, 
                                        "rate1_pair": 'EUR2USD',
                                        "VOL_rate1": VOL_rate1, 
                                        "FD1": FD1,
                                        }, ignore_index=True)

        Final_df_2 = Final_df_2.append({"times_2": dt_2,
                                        "timestamp_2": timestamp_2,
                                        "max_rate2": max_rate2,
                                        "min_rate2": min_rate2, 
                                        "mean_rate2": mean_rate2, 
                                        "rate2_pair": 'EUR2AUD',
                                        "VOL_rate2": VOL_rate2, 
                                        "FD2": FD2,
                                        }, ignore_index=True)

        Final_df_3 = Final_df_3.append({"times_3": dt_3,
                                        "timestamp_3": timestamp_3,
                                        "max_rate3": max_rate3,
                                        "min_rate3": min_rate3, 
                                        "mean_rate3": mean_rate3, 
                                        "rate3_pair": 'EUR2CAD',
                                        "VOL_rate3": VOL_rate3, 
                                        "FD3": FD3,
                                        }, ignore_index=True)
        
        print(Final_df_1)
        print(Final_df_2)
        print(Final_df_3)
    
        
    # 获取更新Keltner Bands
    keltner_upper_bands_1 = []
    keltner_upper_bands_2 = []
    keltner_upper_bands_3 = []
    keltner_lower_bands_1 = []
    keltner_lower_bands_2 = []
    keltner_lower_bands_3 = []
    for n in range(1, 101):
        upper_band_1 = mean_rate1 + n * 0.025 * VOL_rate1
        upper_band_2 = mean_rate2 + n * 0.025 * VOL_rate2
        upper_band_3 = mean_rate3 + n * 0.025 * VOL_rate3
        lower_band_1 = mean_rate1 - n * 0.025 * VOL_rate1
        lower_band_2 = mean_rate2 - n * 0.025 * VOL_rate2
        lower_band_3 = mean_rate3 - n * 0.025 * VOL_rate3
        keltner_upper_bands_1.append(upper_band_1)
        keltner_upper_bands_2.append(upper_band_2)
        keltner_upper_bands_3.append(upper_band_3)
        keltner_lower_bands_1.append(lower_band_1)
        keltner_lower_bands_2.append(lower_band_2)
        keltner_lower_bands_3.append(lower_band_3)
    keltner_bands_1 = keltner_upper_bands_1 + keltner_lower_bands_1
    keltner_bands_2 = keltner_upper_bands_2 + keltner_lower_bands_2
    keltner_bands_3 = keltner_upper_bands_3 + keltner_lower_bands_3


loop= 1
0
loop= 1
1
loop= 1
2
loop= 1
3
loop= 1
4
loop= 1
5
loop= 1
6
loop= 1
7
loop= 1
8
loop= 1
9
loop= 1
10
loop= 1
11
loop= 1
12
loop= 1
13
loop= 1
14
loop= 1
15
loop= 1
16
loop= 1
17
loop= 1
18
loop= 1
19
loop= 1
20
loop= 1
21
loop= 1
22
loop= 1
23
loop= 1
24
loop= 1
25
loop= 1
26
loop= 1
27
loop= 1
28
loop= 1
29
loop= 1
30
loop= 1
31
loop= 1
32
loop= 1
33
loop= 1
34
loop= 1
35
loop= 1
36
loop= 1
37
loop= 1
38
loop= 1
39
loop= 1
40
loop= 1
41
loop= 1
42
loop= 1
43
loop= 1
44
loop= 1
45
loop= 1
46
loop= 1
47
loop= 1
48
loop= 1
49
loop= 1
50
loop= 1
51
loop= 1
52
loop= 1
53
loop= 1
54
loop= 1
55
loop= 1
56
loop= 1
57
loop= 1
58
loop= 1
59
loop= 1
60
loop= 1
61
loop= 1
62
loop= 1
63
loop= 1
64
loop= 1
65
loop= 1
66
loop= 1
67
loop= 1
68
loop= 1
69
loop= 1
70
loop= 1
71
loop= 1
72
loop= 1
73
loop= 1
74
loop= 1
75
loop= 1
76
loop= 1
77
loop= 1
78
loop= 1
79
loop= 1
80
loop= 1
81
loop= 1
82
loop= 1
83
loop= 1
84
loop= 1
85
loop= 1
86
loop= 1
87
loop= 1
88
loop= 1
89
loop= 1
90
loop= 1
9

              times_1    timestamp_1  max_rate1  min_rate1  mean_rate1  \
0 2023-06-06 15:02:53  1686078173000     1.0696     1.0692    1.069413   

  rate1_pair  VOL_rate1           FD1  
0    EUR2USD   0.000374  2.945000e+06  
              times_2     timestamp_  max_rate2  min_rate2  mean_rate2  \
0 2023-06-06 15:02:55  1686078175000    1.60248     1.6016    1.602133   

  rate2_pair  VOL_rate2        FD2  
0    EUR2AUD   0.000549  2037500.0  
              times_3    timestamp_3  max_rate3  min_rate3  mean_rate3  \
0 2023-06-06 15:02:53  1686078173000    1.43327     1.4324    1.433088   

  rate3_pair  VOL_rate3           FD3  
0    EUR2CAD   0.000607  1.721839e+06  
loop= 3
0
loop= 3
1
loop= 3
2
loop= 3
3
loop= 3
4
loop= 3
5
loop= 3
6
loop= 3
7
loop= 3
8
loop= 3
9
loop= 3
10
loop= 3
11
loop= 3
12
loop= 3
13
loop= 3
14
loop= 3
15
loop= 3
16
loop= 3
17
loop= 3
18
loop= 3
19
loop= 3
20
loop= 3
21
loop= 3
22
loop= 3
23
loop= 3
24
loop= 3
25
loop= 3
26
loop= 3
27
loop= 3
28
loop= 3
29

loop= 4
208
loop= 4
209
loop= 4
210
loop= 4
211
loop= 4
212
loop= 4
213
loop= 4
214
loop= 4
215
loop= 4
216
loop= 4
217
loop= 4
218
loop= 4
219
loop= 4
220
loop= 4
221
loop= 4
222
loop= 4
223
loop= 4
224
loop= 4
225
loop= 4
226
loop= 4
227
loop= 4
228
loop= 4
229
loop= 4
230
loop= 4
231
loop= 4
232
loop= 4
233
loop= 4
234
loop= 4
235
loop= 4
236
loop= 4
237
loop= 4
238
loop= 4
239
loop= 4
240
loop= 4
241
loop= 4
242
loop= 4
243
loop= 4
244
loop= 4
245
loop= 4
246
loop= 4
247
loop= 4
248
loop= 4
249
loop= 4
250
loop= 4
251
loop= 4
252
loop= 4
253
loop= 4
254
loop= 4
255
loop= 4
256
loop= 4
257
loop= 4
258
loop= 4
259
loop= 4
260
loop= 4
261
loop= 4
262
loop= 4
263
loop= 4
264
loop= 4
265
loop= 4
266
loop= 4
267
loop= 4
268
loop= 4
269
loop= 4
270
loop= 4
271
loop= 4
272
loop= 4
273
loop= 4
274
loop= 4
275
loop= 4
276
loop= 4
277
loop= 4
278
loop= 4
279
loop= 4
280
loop= 4
281
loop= 4
282
loop= 4
283
loop= 4
284
loop= 4
285
loop= 4
286
loop= 4
287
loop= 4
288
loop= 4
289
loop= 4
290
loop

loop= 6
1
loop= 6
2
loop= 6
3
loop= 6
4
loop= 6
5
loop= 6
6
loop= 6
7
loop= 6
8
loop= 6
9
loop= 6
10
loop= 6
11
loop= 6
12
loop= 6
13
loop= 6
14
loop= 6
15
loop= 6
16
loop= 6
17
loop= 6
18
loop= 6
19
loop= 6
20
loop= 6
21
loop= 6
22
loop= 6
23
loop= 6
24
loop= 6
25
loop= 6
26
loop= 6
27
loop= 6
28
loop= 6
29
loop= 6
30
loop= 6
31
loop= 6
32
loop= 6
33
loop= 6
34
loop= 6
35
loop= 6
36
loop= 6
37
loop= 6
38
loop= 6
39
loop= 6
40
loop= 6
41
loop= 6
42
loop= 6
43
loop= 6
44
loop= 6
45
loop= 6
46
loop= 6
47
loop= 6
48
loop= 6
49
loop= 6
50
loop= 6
51
loop= 6
52
loop= 6
53
loop= 6
54
loop= 6
55
loop= 6
56
loop= 6
57
loop= 6
58
loop= 6
59
loop= 6
60
loop= 6
61
loop= 6
62
loop= 6
63
loop= 6
64
loop= 6
65
loop= 6
66
loop= 6
67
loop= 6
68
loop= 6
69
loop= 6
70
loop= 6
71
loop= 6
72
loop= 6
73
loop= 6
74
loop= 6
75
loop= 6
76
loop= 6
77
loop= 6
78
loop= 6
79
loop= 6
80
loop= 6
81
loop= 6
82
loop= 6
83
loop= 6
84
loop= 6
85
loop= 6
86
loop= 6
87
loop= 6
88
loop= 6
89
loop= 6
90
loop= 6
91
loop= 6


loop= 7
176
loop= 7
177
loop= 7
178
loop= 7
179
loop= 7
180
loop= 7
181
loop= 7
182
loop= 7
183
loop= 7
184
loop= 7
185
loop= 7
186
loop= 7
187
loop= 7
188
loop= 7
189
loop= 7
190
loop= 7
191
loop= 7
192
loop= 7
193
loop= 7
194
loop= 7
195
loop= 7
196
loop= 7
197
loop= 7
198
loop= 7
199
loop= 7
200
loop= 7
201
loop= 7
202
loop= 7
203
loop= 7
204
loop= 7
205
loop= 7
206
loop= 7
207
loop= 7
208
loop= 7
209
loop= 7
210
loop= 7
211
loop= 7
212
loop= 7
213
loop= 7
214
loop= 7
215
loop= 7
216
loop= 7
217
loop= 7
218
loop= 7
219
loop= 7
220
loop= 7
221
loop= 7
222
loop= 7
223
loop= 7
224
loop= 7
225
loop= 7
226
loop= 7
227
loop= 7
228
loop= 7
229
loop= 7
230
loop= 7
231
loop= 7
232
loop= 7
233
loop= 7
234
loop= 7
235
loop= 7
236
loop= 7
237
loop= 7
238
loop= 7
239
loop= 7
240
loop= 7
241
loop= 7
242
loop= 7
243
loop= 7
244
loop= 7
245
loop= 7
246
loop= 7
247
loop= 7
248
loop= 7
249
loop= 7
250
loop= 7
251
loop= 7
252
loop= 7
253
loop= 7
254
loop= 7
255
loop= 7
256
loop= 7
257
loop= 7
258
loop

...

loop= 46
1
loop= 46
2
loop= 46
3
loop= 46
4
loop= 46
5
loop= 46
6
loop= 46
7
loop= 46
8
loop= 46
9
loop= 46
10
loop= 46
11
loop= 46
12
loop= 46
13
loop= 46
14
loop= 46
15
loop= 46
16
loop= 46
17
loop= 46
18
loop= 46
19
loop= 46
20
loop= 46
21
loop= 46
22
loop= 46
23
loop= 46
24
loop= 46
25
loop= 46
26
loop= 46
27
loop= 46
28
loop= 46
29
loop= 46
30
loop= 46
31
loop= 46
32
loop= 46
33
loop= 46
34
loop= 46
35
loop= 46
36
loop= 46
37
loop= 46
38
loop= 46
39
loop= 46
40
loop= 46
41
loop= 46
42
loop= 46
43
loop= 46
44
loop= 46
45
loop= 46
46
loop= 46
47
loop= 46
48
loop= 46
49
loop= 46
50
loop= 46
51
loop= 46
52
loop= 46
53
loop= 46
54
loop= 46
55
loop= 46
56
loop= 46
57
loop= 46
58
loop= 46
59
loop= 46
60
loop= 46
61
loop= 46
62
loop= 46
63
loop= 46
64
loop= 46
65
loop= 46
66
loop= 46
67
loop= 46
68
loop= 46
69
loop= 46
70
loop= 46
71
loop= 46
72
loop= 46
73
loop= 46
74
loop= 46
75
loop= 46
76
loop= 46
77
loop= 46
78
loop= 46
79
loop= 46
80
loop= 46
81
loop= 46
82
loop= 46
83
loop= 46
84
l

loop= 47
1
loop= 47
2
loop= 47
3
loop= 47
4
loop= 47
5
loop= 47
6
loop= 47
7
loop= 47
8
loop= 47
9
loop= 47
10
loop= 47
11
loop= 47
12
loop= 47
13
loop= 47
14
loop= 47
15
loop= 47
16
loop= 47
17
loop= 47
18
loop= 47
19
loop= 47
20
loop= 47
21
loop= 47
22
loop= 47
23
loop= 47
24
loop= 47
25
loop= 47
26
loop= 47
27
loop= 47
28
loop= 47
29
loop= 47
30
loop= 47
31
loop= 47
32
loop= 47
33
loop= 47
34
loop= 47
35
loop= 47
36
loop= 47
37
loop= 47
38
loop= 47
39
loop= 47
40
loop= 47
41
loop= 47
42
loop= 47
43
loop= 47
44
loop= 47
45
loop= 47
46
loop= 47
47
loop= 47
48
loop= 47
49
loop= 47
50
loop= 47
51
loop= 47
52
loop= 47
53
loop= 47
54
loop= 47
55
loop= 47
56
loop= 47
57
loop= 47
58
loop= 47
59
loop= 47
60
loop= 47
61
loop= 47
62
loop= 47
63
loop= 47
64
loop= 47
65
loop= 47
66
loop= 47
67
loop= 47
68
loop= 47
69
loop= 47
70
loop= 47
71
loop= 47
72
loop= 47
73
loop= 47
74
loop= 47
75
loop= 47
76
loop= 47
77
loop= 47
78
loop= 47
79
loop= 47
80
loop= 47
81
loop= 47
82
loop= 47
83
loop= 47
84
l

loop= 48
1
loop= 48
2
loop= 48
3
loop= 48
4
loop= 48
5
loop= 48
6
loop= 48
7
loop= 48
8
loop= 48
9
loop= 48
10
loop= 48
11
loop= 48
12
loop= 48
13
loop= 48
14
loop= 48
15
loop= 48
16
loop= 48
17
loop= 48
18
loop= 48
19
loop= 48
20
loop= 48
21
loop= 48
22
loop= 48
23
loop= 48
24
loop= 48
25
loop= 48
26
loop= 48
27
loop= 48
28
loop= 48
29
loop= 48
30
loop= 48
31
loop= 48
32
loop= 48
33
loop= 48
34
loop= 48
35
loop= 48
36
loop= 48
37
loop= 48
38
loop= 48
39
loop= 48
40
loop= 48
41
loop= 48
42
loop= 48
43
loop= 48
44
loop= 48
45
loop= 48
46
loop= 48
47
loop= 48
48
loop= 48
49
loop= 48
50
loop= 48
51
loop= 48
52
loop= 48
53
loop= 48
54
loop= 48
55
loop= 48
56
loop= 48
57
loop= 48
58
loop= 48
59
loop= 48
60
loop= 48
61
loop= 48
62
loop= 48
63
loop= 48
64
loop= 48
65
loop= 48
66
loop= 48
67
loop= 48
68
loop= 48
69
loop= 48
70
loop= 48
71
loop= 48
72
loop= 48
73
loop= 48
74
loop= 48
75
loop= 48
76
loop= 48
77
loop= 48
78
loop= 48
79
loop= 48
80
loop= 48
81
loop= 48
82
loop= 48
83
loop= 48
84
l

loop= 49
1
loop= 49
2
loop= 49
3
loop= 49
4
loop= 49
5
loop= 49
6
loop= 49
7
loop= 49
8
loop= 49
9
loop= 49
10
loop= 49
11
loop= 49
12
loop= 49
13
loop= 49
14
loop= 49
15
loop= 49
16
loop= 49
17
loop= 49
18
loop= 49
19
loop= 49
20
loop= 49
21
loop= 49
22
loop= 49
23
loop= 49
24
loop= 49
25
loop= 49
26
loop= 49
27
loop= 49
28
loop= 49
29
loop= 49
30
loop= 49
31
loop= 49
32
loop= 49
33
loop= 49
34
loop= 49
35
loop= 49
36
loop= 49
37
loop= 49
38
loop= 49
39
loop= 49
40
loop= 49
41
loop= 49
42
loop= 49
43
loop= 49
44
loop= 49
45
loop= 49
46
loop= 49
47
loop= 49
48
loop= 49
49
loop= 49
50
loop= 49
51
loop= 49
52
loop= 49
53
loop= 49
54
loop= 49
55
loop= 49
56
loop= 49
57
loop= 49
58
loop= 49
59
loop= 49
60
loop= 49
61
loop= 49
62
loop= 49
63
loop= 49
64
loop= 49
65
loop= 49
66
loop= 49
67
loop= 49
68
loop= 49
69
loop= 49
70
loop= 49
71
loop= 49
72
loop= 49
73
loop= 49
74
loop= 49
75
loop= 49
76
loop= 49
77
loop= 49
78
loop= 49
79
loop= 49
80
loop= 49
81
loop= 49
82
loop= 49
83
loop= 49
84
l

loop= 50
1
loop= 50
2
loop= 50
3
loop= 50
4
loop= 50
5
loop= 50
6
loop= 50
7
loop= 50
8
loop= 50
9
loop= 50
10
loop= 50
11
loop= 50
12
loop= 50
13
loop= 50
14
loop= 50
15
loop= 50
16
loop= 50
17
loop= 50
18
loop= 50
19
loop= 50
20
loop= 50
21
loop= 50
22
loop= 50
23
loop= 50
24
loop= 50
25
loop= 50
26
loop= 50
27
loop= 50
28
loop= 50
29
loop= 50
30
loop= 50
31
loop= 50
32
loop= 50
33
loop= 50
34
loop= 50
35
loop= 50
36
loop= 50
37
loop= 50
38
loop= 50
39
loop= 50
40
loop= 50
41
loop= 50
42
loop= 50
43
loop= 50
44
loop= 50
45
loop= 50
46
loop= 50
47
loop= 50
48
loop= 50
49
loop= 50
50
loop= 50
51
loop= 50
52
loop= 50
53
loop= 50
54
loop= 50
55
loop= 50
56
loop= 50
57
loop= 50
58
loop= 50
59
loop= 50
60
loop= 50
61
loop= 50
62
loop= 50
63
loop= 50
64
loop= 50
65
loop= 50
66
loop= 50
67
loop= 50
68
loop= 50
69
loop= 50
70
loop= 50
71
loop= 50
72
loop= 50
73
loop= 50
74
loop= 50
75
loop= 50
76
loop= 50
77
loop= 50
78
loop= 50
79
loop= 50
80
loop= 50
81
loop= 50
82
loop= 50
83
loop= 50
84
l

In [4]:
Final_df_1

,times_1,timestamp_1,max_rate1,min_rate1,mean_rate1,rate1_pair,VOL_rate1,FD1
0,2023-06-06 15:02:53,1686078173000,1.06960,1.06920,1.069413,EUR2USD,0.000374,2.945000e+06
1,2023-06-06 15:08:55,1686078535000,1.06940,1.06910,1.069295,EUR2USD,0.000281,3.793333e+06
2,2023-06-06 15:14:53,1686078893000,1.06931,1.06910,1.069247,EUR2USD,0.000196,4.171429e+06
3,2023-06-06 15:20:55,1686079255000,1.06946,1.06900,1.069336,EUR2USD,0.000430,3.402174e+06
4,2023-06-06 15:26:58,1686079618000,1.06962,1.06930,1.069498,EUR2USD,0.000299,3.150000e+06
5,2023-06-06 15:32:55,1686079975000,1.06954,1.06930,1.069458,EUR2USD,0.000224,3.816667e+06
6,2023-06-06 15:39:00,1686080340000,1.06948,1.06890,1.069220,EUR2USD,0.000542,2.270690e+06
7,2023-06-06 15:45:01,1686080701000,1.06932,1.06890,1.069166,EUR2USD,0.000393,1.511905e+06
8,2023-06-06 15:51:02,1686081062000,1.06937,1.06910,1.069276,EUR2USD,0.000253,3.111111e+06
9,2023-06-06 15:57:03,1686081423000,1.06936,1.06900,1.069236,EUR2USD,0.000337,3.716667e+06


## Store data into CSV

In [5]:
Final_df_1.to_csv('Final_df_1.csv', index=False)
Final_df_2.to_csv('Final_df_2.csv', index=False)
Final_df_3.to_csv('Final_df_3.csv', index=False)

## Store data into SQLite

In [6]:
# 连接到SQLite数据库
conn = sqlite3.connect('Final_df_1.db')
# 将DataFrame写入SQLite数据库中的表
Final_df_1.to_sql('Final_df_1', conn, if_exists='replace', index=False)
# 关闭数据库连接
conn.close()

In [7]:
# 连接到SQLite数据库
conn = sqlite3.connect('Final_df_2.db')
# 将DataFrame写入SQLite数据库中的表
Final_df_2.to_sql('Final_df_2', conn, if_exists='replace', index=False)
# 关闭数据库连接
conn.close()

In [8]:
# 连接到SQLite数据库
conn = sqlite3.connect('Final_df_3.db')
# 将DataFrame写入SQLite数据库中的表
Final_df_3.to_sql('Final_df_3', conn, if_exists='replace', index=False)
# 关闭数据库连接
conn.close()

## Store data into MongoDB

In [9]:
import pymongo
from pymongo import MongoClient

In [10]:
client = MongoClient('mongodb://localhost:27017/')
db = client['DATA_ENGINEERING_HW2']
collection = db['Final_df_1']
data = Final_df_1.to_dict(orient='records')
collection.insert_many(data)

In [11]:
client = MongoClient('mongodb://localhost:27017/')
db = client['DATA_ENGINEERING_HW2']
collection = db['Final_df_2']
data = Final_df_2.to_dict(orient='records')
collection.insert_many(data)

In [12]:
client = MongoClient('mongodb://localhost:27017/')
db = client['DATA_ENGINEERING_HW2']
collection = db['Final_df_3']
data = Final_df_3.to_dict(orient='records')
collection.insert_many(data)

In [13]:
database_names = client.list_database_names()
if 'DATA_ENGINEERING_HW2' in database_names:
    print("Database exists")

Database exists


In [14]:
collection_names = db.list_collection_names()
if 'Final_df_1' in collection_names:
    print("Collection exists")

Collection exists


In [15]:
collection_names = db.list_collection_names()
if 'Final_df_2' in collection_names:
    print("Collection exists")

Collection exists


In [16]:
collection_names = db.list_collection_names()
if 'Final_df_3' in collection_names:
    print("Collection exists")

Collection exists
